# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '../../../../notebooks/' not in sys.path:
    sys.path.append('../../../../notebooks/')
if 'src' not in sys.path:
    sys.path.insert(0, 'src')

import utils.constants as cts

from data_loaders.data_loader import DLName
from gt_loaders.gt_names import GTName
from exp_runner import ExperimentRunner
from base.model_evaluator import DataSource, DataPredSelection
from base.base_models import BaseModel
from base.optimizers import Optimizer
from base.model_creator import MTLApproach, NAS_MTLApproach

 ==> Restrict GPU memory growth: True


# Start Network runner

In [3]:
kwargs = { 
    'use_neptune': True,
    'exp_params' : {
        'name': 'neural_arch_search',
        'description': 'First train with neural architecture search',
        'tags': ['ground truths', 'nas', 'nas_approach_1'],
        'src_files': ["src/**/*.py"]
    },
    'properties': {
        'approach': NAS_MTLApproach.APPROACH_1,
        'reqs': list(cts.ICAO_REQ),
        'aligned': True,
        'use_gt_data': True,
        'gt_names': {
            'train_validation': [],
            'test': [],
            'train_validation_test': [GTName.FVC]
        },
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': False,
        'orig_model_experiment_id': '',
        'sample_training_data': False,
        'sample_prop': 1.0
    },
    'net_train_params': {
        'base_model': BaseModel.VGG16,
        'batch_size': 32,
        'n_epochs': 10,
        'early_stopping': 10,
        'learning_rate': 1e-3,
        'optimizer': Optimizer.ADAMAX,
        'dropout': 0.3
    },
    'nas_params': {
        'max_blocks_per_branch': 5,
        'n_epochs': 2,
        'n_trials': 3
    }
}

runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 98268
Process ID: 120949
---------------------------
-----
Use Neptune:  True
-----
-------------------
Args: 
{'exp_params': {'description': 'First train with neural architecture search',
                'name': 'neural_arch_search',
                'src_files': ['src/**/*.py'],
                'tags': ['ground truths', 'nas', 'nas_approach_1']},
 'nas_params': {'max_blocks_per_branch': 5, 'n_epochs': 2, 'n_trials': 3},
 'net_train_params': {'base_model': <BaseModel.VGG16: {'target_size': (224, 224), 'prep_function': <function preprocess_input at 0x7f399f6773a0>}>,
                      'batch_size': 32,
                      'dropout': 0.3,
                      'early_stopping': 10,
                      'learning_rate': 0.001,
                      'n_epochs': 10,
                      'optimizer': <Optimizer.ADAMAX: 'Adamax'>},
 'properties': {'aligned': True,
            

# Load Data

In [4]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
Loading GT FVC - TRAIN split...
..Ignoring 0 empty label values
Input data.shape: (4926, 26)
Loading GT FVC - VALIDATION split...
..Ignoring 0 empty label values
Input data.shape: (547, 26)
Loading GT FVC - TEST split...
..Ignoring 0 empty label values
Input data.shape: (288, 26)
Data loaded


# Producing Fake Data

In [5]:
runner.produce_fake_data()

-------------------- producing fake data for experimental purposes -------------------
fake_train_data.shape: (500, 26)
fake_validation_data_df.shape: (100, 26)
fake_test_data_df.shape: (50, 26)


# Data Generators

In [6]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 500 validated image filenames.
Found 100 validated image filenames.
Found 50 validated image filenames.
TOTAL: 650

Logging class indices
 .. MTL model not logging class indices!

Logging class labels
 COMPLIANT label: 1
 NON_COMPLIANT label: 0
 DUMMY label: -1
 DUMMY_CLS label: 2
 NO_ANSWER label: -99


# Neural Architecture Search

In [7]:
runner.run_neural_architeture_search()

-------------------- run neural architecture search -------------------
++++++++++++++++++++ STARTING NEW TRAIN ++++++++++++++++++++
 ----- Training 1 | Config: {'n_denses_0': 3, 'n_denses_1': 2, 'n_denses_2': 2, 'n_denses_3': 1} --------
Creating model...
Model created
Training VGG16 network
 .. Not fine tuning base model...
  .. Total params: 15,572,654
  .. Trainable params: 857,966
  .. Non-trainable params: 14,714,688
Epoch 1/2


15/15 [==============================] - 23s 707ms/step - loss: 2.7992 - background_loss: 0.7738 - close_loss: 0.9565 - ink_mark_loss: 1.0810 - pixelation_loss: 1.1603 - washed_out_loss: 0.7159 - blurred_loss: 0.8378 - sh_head_loss: 1.0357 - mouth_loss: 0.9537 - veil_loss: 0.8129 - red_eyes_loss: 1.2218 - flash_lenses_loss: 1.1147 - dark_glasses_loss: 1.2839 - l_away_loss: 0.9203 - frame_eyes_loss: 1.3481 - hair_eyes_loss: 1.1097 - eyes_closed_loss: 1.3436 - frames_heavy_loss: 1.0337 - sh_face_loss: 1.3349 - skin_tone_loss: 2.5032 - light_loss: 1.1068 - hat_loss: 1.4037 - rotation_loss: 2.7569 - reflection_loss: 1.1829 - background_accuracy: 0.5128 - close_accuracy: 0.5000 - ink_mark_accuracy: 0.4765 - pixelation_accuracy: 0.4872 - washed_out_accuracy: 0.5427 - blurred_accuracy: 0.5064 - sh_head_accuracy: 0.5278 - mouth_accuracy: 0.4936 - veil_accuracy: 0.4957 - red_eyes_accuracy: 0.5299 - flash_lenses_accuracy: 0.5299 - dark_glasses_accuracy: 0.5491 - l_away_accuracy: 0.4615 - frame_e

# Finish Neptune

In [8]:
runner.finish_experiment()

-------------------- finish experiment -------------------
Finishing Neptune


# Testing Network Modification